# Meta-stability

Start initial paper extensions of Rick meta stabililty. 

## What is metastability?
Meta-stability refers to the system being able to switch between stable states. Think of it as energy wells, the wells can be global or local but the system can be stuck in a metastable state for some time. 

- In the original draft a locally-tree like graph was used. Followed by a degree-based analysis on the probability of flipping of a nodes.  However as will be shown below, meta-stability occurs in lattice graphs as well. This means that the degree approach will not generalize well to other types of graphs. 

In the original draft metastability was hypothesized to be caused by putting the nodes in a system along a singular axis. On one end of the axis a node was mainly locally correlated with the system, i.e. it mainly processes the noise in the system. On the other end of the spectrum there are nodes that are mainly associated with long-term behavior of the system. 

## What is the goal of this paper?
At its core, we know that the Ising model has a rule that interprets noise and allows for nodal behavior from which noise is combined with correlated behavior. The interaction between noise and this rule allows the system to be driven to one the one hand pattern formation and synchronization, and on the other purely uniform behavior. There is no **mechanism** to be accounted for besides this. Mapping this to some information plane, will provide no further insights. From the master equation it can be hypothesized that noise will be amplified for non-zero temperatures. As the temperature approaches infinity, the noise amplification will be linearized, i.e. the effect of noise will be both higher (y-offset), and linear as a function of input energy for a fixed degree for every node. 

Doing this mapping for a network with a difference in nodal degree, just alters the possible states a node can be in and whether it is linear or not with noise. However, I feel like this is just a mathematical exercise without providing any novel insights into a property of the kinetic Ising model. 

Then again writing this (in my opinion trivial result) down could be beneficial as I cannot recall a paper that goes into depth unravelling this behavior. By not pretending it is in any way novel is required and essential. 

### Conclusions
There are some questions regarding the goal of the noise induced dynamical transitions. If the goal is to look for differential contribution by structural differences, then you will find those. However, this will not yield any understanding regarding the tipping points that occur. The tipping points occur due to random influences at a local scale that 'play out' in favor of one of the borders of average magnetization. 

As a consequence, it is also senseless to ask questions about what properties differentiate these nodes, i.e. what makes the contribution of these nodes different. For example on a lattice, the tipping points also occur. These nodes however have the same structural connectivity. In addition, according to ergodic theorem as we sample into infinity, the information decay curves $I(s_i^{t\pm\delta}:S^T)$ where $S^{t}$ is the state distribution at tipping point, will also yield to collapse of the nodal information to be the same over time. 

On different graph structures one could find some property that yields different information content, but they do not reflect any insights into why the tipping points occur in the Ising model. What is needed is a meaningful tipping point, i.e. a tipping that occurs in the dynamics not caused through stochastic noise. 


In [1]:
from plexsim.models import *
import src.toolbox

from imi.toolbox import infcy
print(imi.toolbox.infcy)
from IMI.toolbox import infcy
# from information_impact import Utils
from information_impact.Utils.graph import recursive_tree
import plotly.graph_objects as go, numpy as np, networkx as nx
from scipy import optimize, interpolate, ndimage
n = 30
# g = nx.grid_graph([n, n], True)
# g = nx.path_graph(n)
# g = nx.cycle_graph(n)
g = recursive_tree(5)
m = Potts(g, sampleSize = 1)

temps = np.linspace(0, 4, 100)
sim = m.magnetize(temps, n = int(1e5))
sim[0] = ndimage.gaussian_filter1d(sim[0], 3)

ModuleNotFoundError: No module named 'imi.toolbox'

In [ ]:
import plotly.express as px
def sig(x, a, b, c):
    return a / (1 + b*np.exp(c * x))
fit, _ = optimize.curve_fit(sig, temps, sim[0])
x      = np.linspace(0, 5)
mini = lambda x, fit, theta: abs(sig(x, *fit) - theta)
theta = .5
a = optimize.fmin(mini, 1, args = (fit, theta))[0]
print(a)
line = go.Scatter(x = temps, y = sim[0])
spline = go.Scatter(x = x, y = sig(x, *fit))
fig = go.Figure(data = [line, spline])
fig.add_shape(\
             type = 'line',\
             x0   = a,\
             x1   = a,\
             y0   = 0,\
             y1   = 1,\
             line = dict(color = "black"))
fig.show()

In [ ]:
# set to T optimized point and simulate
m.t = a
m.reset()
# m.states = 1
# m.states[0] = 1
# target = '(0, 0)'
# for idx, i in enumerate(nx.shortest_path(m.graph, target)):
#     if idx <= 10:
#         m.states[m.mapping[i]] = 0
time_signal = m.simulate(int(1e5))


In [ ]:
import sklearn.decomposition
# mean = np.real(np.exp(2 * np.pi * np.complex(0, 1) * time_signal))
mean = time_signal.mean(1)
filtered = ndimage.gaussian_filter1d(mean, 100) # filter out noise

idx  = np.where(filtered < .5)[0]
diff = np.ones(mean.shape)
diff[idx] = 0
switchidx = np.diff(diff)
deltas = np.zeros(mean.shape) 
deltas[np.where(switchidx == 1)[0]] = 1
switcher = np.where(abs(switchidx) == 1)[0]

In [ ]:
line = go.Scatter(y = mean, name = 'original trace')
dif = go.Scatter(y = diff, name = 'Delta for switches')
f   = go.Scatter(y = filtered, name = 'Gaussian filtered')
g    = go.Scatter(y = deltas, name = 'deltas')
# h    = go.Scatter(y = tmp, name = 'hallo')
fig = go.Figure(data = [line, dif, f, g])
fig.update_layout(\
                  title = 'Potts',\
                 xaxis = dict(title = 'time'),\
                 yaxis = dict(title = 'mean magnetization'))
fig.show()

In [ ]:
x = np.linspace(-4, 4)
p = lambda x, b: 1/(1 + np.exp(-x * beta))
betas = np.linspace(.001, 1, 5)
beta  = 1/temps

lines = []
for beta in betas:
    line = go.Scatter(x = x, y = p(x, beta),\
                     name = 1/beta)
    lines.append(line)
fig = go.Figure(data = lines)
fig.update_layout(yaxis_title = 'P(s = 1 | x)', xaxis_title = 'x')
fig.show()

For a lattice graph, degree does not make a difference to the switching probability. The only difference would be the local energy that a nodes obtains. This depends on the initial conditions and the temporal dynamics over time. Local frustrations occur randomly and this noise is transmitted. The succes of noise transmission would be dependendent on the susceptibility of the nodes which is dependent on the temperature of the system. The higher the temperature, the smaller the beta value, and the more linear the node will transmit noise.  

This can be seen by the fact that for lower beta (above) the curve is flatter. Consequently, if a node were to receive arround 0 energy, the flip probability will remain around .5 as the energy input diverges from 0, for higher beta (lower temperature), the node tends to favor one or the other state as the energy diverges from zero. 



In [ ]:

from jupyter_plotly_dash import JupyterDash
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

tmp = []
window = 1

for i in switcher:
    for j in range(-window, window):
        tmp.append(i + j)
# print(len(switcher), len(tmp))
DO = np.array(tmp)
app = JupyterDash('SimpleExample')
app.layout = html.Div([
    html.Div(id = 'app_test'),\
    html.Div([
        dcc.Slider(
        id   = 'time-slider',
        min  = 0,
        max  = len(tmp) - 1,
        step = 1,
        value= 0,
        updatemode =  'drag'),\
    dcc.Graph(id = 'animation'),\
    ],
    ),
    html.Div([

    ],\
        style = dict(width = 50, height = 50,\
                      margin = 'auto')),\
    html.Div(id = 'time-slider-output')

])

from plotly import subplots
colors = px.colors.sequential.Magenta
idx    = np.linspace(0, len(colors) - 1, len(m.agentStates), dtype = int)
# print(idx)
colors = [(j / (m.nStates-1), np.array(colors)[i]) for j, i in  enumerate(idx)]

hm = go.Heatmap( z = time_signal[0].reshape(n, n), \
                colorscale = colors,\
                zmin = m.agentStates[0],\
                zmax = m.agentStates[-1],\
               )
# print(DO)
fig = go.Figure(data = hm)
fig.update_layout( template = 'plotly_white', \
                 autosize = False, width = 600, height = 600)

@app.callback(
    Output('animation', 'figure'),
    [Input('time-slider', 'value')])
def update_output(value):
    try:
        for d in fig.data:
            d.z = time_signal[DO[value]].reshape(n, n)
            fig.update_layout(title = f'{DO[value]}')
    except Exception as e:
        print(e)
    return fig 
app

In [ ]:
import sys; sys.path.insert(0, '../')
from Toolbox import infcy
d = time_signal[DO[::2]]
s = {tuple(i): 1/len(d) for i in d}
print(len(s))
sim = infcy.Simulator(m)
s, c =  sim.forward(s, \
                   repeats = int(1e4),\
                   time_steps  = 5).values()
print(type(c), type(s))
_, mi = infcy.mutualInformation(c, s)



In [ ]:
lines = []
for i in range(m.nNodes):
    line = go.Scatter(y = mi[:, i], name = m.rmapping[i])
    lines.append(line)
fig = go.Figure(data = lines)
fig.show()

In [ ]:
nx.draw(m.graph, with_labels = 1)

# What happens at these transitions? 
Given that the degree is not important, we wonder what happens at these transitions points. Why does the transition occur? Let's look at two cases

Define $T_0$ the time at transition
- What distribution does the system go into as we repeat from this state forwards?
   - Does it always flip? --> Probably not as it it a probabilistic model. We would find in the limit the transition ratios given by the node update rule.
- How big of a block does the system need to have to switch?
    - Start with all nodes at state = 1, then look for time of switch to occur.
    - Slowly introduce a block with nodes = 0, then look for the time of switch to occur
  

# Tipping points information decomposition
- Get a N tipping points
- Plot IMI as a function of T



In [ ]:
from Toolbox import infcy

from pyprind import ProgBar
def capture_tipping(model, n_samples, N, window_size):
#     buffer = model.simulate(window_size * 2)
    buffer = m.simulate(n_samples + 2 * window_size)
    stores = {}
    n = 0
    j = 0
    pbar = ProgBar(n_samples)
    for ni in range(N):
#         buffer[:-1] = buffer[1:]
        buffer  = m.simulate(n_samples + 2 * window_size)
        # assert tipping point
        tipps = np.diff(np.sign(buffer.mean(1) - .5))
#         print(abs(tipps), buffer.mean(1) - .5)
        
        idx = np.argwhere(abs(tipps) > 0)
        # located:
        #    center the buffer
        pbar.update(1)
        if len(idx):
            idx = idx[0]
            pbar.update(1)
#             print(idx)
            for i in idx:
                if i <= window_size or i + window_size >= n_samples:
                    continue
                else:
                    state = tuple(buffer[i].tolist())
                    stores[state] = stores.get(state, []) + \
                    [buffer[i - window_size: i + window_size, :]]
        
    return stores
                

# print(m.simulate(10).mean(1))
m.t = 5
sim = infcy.Simulator(m)
n_samples = int(1e5)
window    = 50
center    = True
tipps = capture_tipping(m, n_samples, 100, window)

# Switch time 
   - Start with all nodes at state = 1, then look for time of switch to occur.
   - Slowly introduce a block with nodes = 0, then look for the time of switch to occur

In [ ]:
nTrials = 50
# neighbors = np.arange(8)
neighbors = np.array([5, 10, 30])
timings = np.zeros((neighbors.size, nTrials))
target = '(0, 0)'
for idx, nei in enumerate(neighbors):
    for trial in range(nTrials):
        m.states = 1
        for jdx, node in enumerate(nx.shortest_path(m.graph, source = target)):
            m.states[m.mapping[node]] = 0
            if jdx >= nei:
                break
            
        res      = m.simulate(int(1e5)).mean(1)
        filtered = ndimage.gaussian_filter(res, 100)
        try:
            zdx = np.where(filtered < .5)[0][0]
        except:
            zdx = np.nan
        timings[idx, trial] = zdx
    if idx:
        print(idx, end = ' ')


In [ ]:
from scipy.stats import sem
scatter = go.Scatter(x = neighbors, \
                     y = np.nanmean(timings, 1), \
                     error_y = dict(array = np.nanstd(timings, axis = 1)\
                                   )\
                    )
fig = go.Figure(data = scatter)
fig.show()

# Probability of switching [with correations from original draft]

The fraction of nodes in state $+1$ is defined as

$$ p^t = \frac{(N + M^t)}{2N}$$
original draft stated

$$p^t = \frac{(N + M^t)}{2}$$ but this is not normalized...

with $M^t = \sum_i^N x_i^t, x_i^t \in \{-1, 1\}$. 

Next the assumption is made that for any node, this fraction get carried and as such the number of nodes in +1 can be written as

$$ E(x_i^t) = p_i^t k_i$$ 


Therefore the switch probabililty for a node in the kinetic Ising model can be written  down

$$ P(x_i^{t+1} = -x_i^t | X^t) = \frac{ \exp(-\beta E(-x_i^t, X^t) }{ \exp(-\beta E(-x_i^t, X^t) + \exp(-\beta E(x_i^t, x^t)} = \frac{1}{ 1 + \exp(\beta k_i (2 p_i^t - 1))}$$

Note that this assumption only works for mean-field, but then the whole idea of having degree is gone. 


In [ ]:
import numpy as np
from plotly.subplots import make_subplots
from plotly import graph_objects as go
degrees = np.linspace(1, 100, 100, dtype = int)
beta = np.linspace(0, 5, 10)
fractions = np.linspace(0, 1, 100)

x, y, z = np.meshgrid(degrees, beta, fractions)
p = 1/ (1 + np.exp(x * (2 * z - 1) / y))
print(p.shape)
figs = make_subplots(rows = 1,\
                    cols = beta.size, \
            shared_xaxes = True,\
            shared_yaxes = True, \
            x_title = 'Degree',\
            y_title = 'Fraction',\
            horizontal_spacing = .0,\
#             vertical_spacing = 0,\
                )
cmin = p.min()
cmax = p.max()

print(x.shape)
for idx in range(beta.size):
    hm = go.Heatmap(\
#                     x = x[idx], 
#                     y = z[idx],\
                    z = p[idx], zmin = cmin, zmax = cmax,\
#                    xaxis = dict(title = 'hello'),\
                   )
    figs.add_trace(hm, col = idx + 1, row = 1)
figs.show()
# p = 1 / np.exp(beta * degre * (2 * fract - 1))